In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score # 정확도 함수
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


In [15]:
Grandmaster = pd.read_csv('../Dataset/perMinuteDataset/5min/CHALLENGER.csv')


In [16]:
win_Grandmaster = Grandmaster[['Diff_FirstBLOOD', 'Diff_FirstDRAGON',
       'Diff_FirstHERALD', 'Diff_Firsttower', 'dragonType', 'WIN_invadeKill', 'WIN_invadeDeath','WIN_controlWARDPlaced', 
       'WIN_Kill_top', 'WIN_Kill_jgl', 'WIN_Kill_mid', 'WIN_Kill_ad', 'WIN_Kill_sup',
       'WIN_Death_top', 'WIN_Death_jgl', 'WIN_Death_mid', 'WIN_Death_ad', 'WIN_Death_sup', 
       'WIN_Asisst_top', 'WIN_Asisst_jgl', 'WIN_Asisst_mid', 'WIN_Asisst_ad', 'WIN_Asisst_sup',
       'WIN_LV_top', 'WIN_LV_jgl', 'WIN_LV_mid', 'WIN_LV_ad', 'WIN_LV_sup',
       'WIN_CS_top', 'WIN_CS_jgl', 'WIN_CS_mid', 'WIN_CS_ad', 'WIN_CS_sup',
       'WIN_jglCS_top', 'WIN_jglCS_jgl', 'WIN_jglCS_mid', 'WIN_jglCS_ad', 'WIN_jglCS_sup',
       'WIN_GOLD_top', 'WIN_GOLD_jgl', 'WIN_GOLD_mid', 'WIN_GOLD_ad', 'WIN_GOLD_sup',
       'WIN_WARDkill', 'WIN_Inhibitor','WIN_TOWERkill', 'WIN_WARDplaced']]
lose_Grandmaster = Grandmaster[['Diff_FirstBLOOD', 'Diff_FirstDRAGON',
       'Diff_FirstHERALD', 'Diff_Firsttower', 'dragonType',
       'LOSE_invadeDeath', 'LOSE_invadeKill',
       'LOSE_controlWARDPlaced',
       'LOSE_Kill_top', 'LOSE_Kill_jgl', 'LOSE_Kill_mid', 'LOSE_Kill_ad', 'LOSE_Kill_sup',
       'LOSE_Death_top', 'LOSE_Death_jgl',
       'LOSE_Death_mid', 'LOSE_Death_ad', 'LOSE_Death_sup',
       'LOSE_Asisst_top', 'LOSE_Asisst_jgl', 'LOSE_Asisst_mid',
       'LOSE_Asisst_ad', 'LOSE_Asisst_sup',
       'LOSE_LV_top', 'LOSE_LV_jgl',
       'LOSE_LV_mid', 'LOSE_LV_ad', 'LOSE_LV_sup',
       'LOSE_CS_top', 'LOSE_CS_jgl',
       'LOSE_CS_mid', 'LOSE_CS_ad', 'LOSE_CS_sup',
       'LOSE_jglCS_top', 'LOSE_jglCS_jgl', 'LOSE_jglCS_mid', 'LOSE_jglCS_ad', 'LOSE_jglCS_sup',
       'LOSE_GOLD_top', 'LOSE_GOLD_jgl',
       'LOSE_GOLD_mid', 'LOSE_GOLD_ad', 'LOSE_GOLD_sup',
       'LOSE_WARDkill', 'LOSE_Inhibitor',
       'LOSE_TOWERkill', 'LOSE_WARDplaced']]
colName = 'WIN'
win_Grandmaster = win_Grandmaster.rename(columns={f'{colName}_invadeKill': 'invadeKill', f'{colName}_invadeDeath': 'invadeDeath', 
                                                  f'{colName}_controlWARDPlaced': 'controlWARDPlaced',
                                                  f'{colName}_Kill_top': 'Kill_top',f'{colName}_Kill_jgl': 'Kill_jgl',f'{colName}_Kill_mid': 'Kill_mid',f'{colName}_Kill_ad': 'Kill_ad', f'{colName}_Kill_sup': 'Kill_sup',
                                                  f'{colName}_Death_top': 'Death_top',f'{colName}_Death_jgl': 'Death_jgl',f'{colName}_Death_mid': 'Death_mid',f'{colName}_Death_ad': 'Death_ad',f'{colName}_Death_sup': 'Death_sup',
                                                  f'{colName}_Asisst_top': 'Assist_top',f'{colName}_Asisst_jgl': 'Assist_jgl',f'{colName}_Asisst_mid': 'Assist_mid',f'{colName}_Asisst_ad': 'Assist_ad',f'{colName}_Asisst_sup': 'Assist_sup',
                                                  f'{colName}_LV_top': 'LV_top',f'{colName}_LV_jgl': 'LV_jgl',f'{colName}_LV_mid': 'LV_mid',f'{colName}_LV_ad': 'LV_ad',f'{colName}_LV_sup': 'LV_sup',
                                                  f'{colName}_CS_top': 'CS_top',f'{colName}_CS_jgl': 'CS_jgl',f'{colName}_CS_mid': 'CS_mid',f'{colName}_CS_ad': 'CS_ad',f'{colName}_CS_sup': 'CS_sup',
                                                  f'{colName}_jglCS_top': 'jglCS_top',f'{colName}_jglCS_jgl': 'jglCS_jgl',f'{colName}_jglCS_mid': 'jglCS_mid',f'{colName}_jglCS_ad': 'jglCS_ad',f'{colName}_jglCS_sup': 'jglCS_sup',
                                                  f'{colName}_GOLD_top': 'GOLD_top',f'{colName}_GOLD_jgl': 'GOLD_jgl',f'{colName}_GOLD_mid': 'GOLD_mid',f'{colName}_GOLD_ad': 'GOLD_ad',f'{colName}_GOLD_sup': 'GOLD_sup',
                                                  f'{colName}_WARDkill': 'WARDkill',f'{colName}_Inhibitor': 'Inhibitor',f'{colName}_TOWERkill': 'TOWERkill',f'{colName}_WARDplaced': 'WARDplaced'})

colName = 'LOSE'
lose_Grandmaster = lose_Grandmaster.rename(columns={f'{colName}_invadeKill': 'invadeKill', f'{colName}_invadeDeath': 'invadeDeath', 
                                                  f'{colName}_controlWARDPlaced': 'controlWARDPlaced',
                                                  f'{colName}_Kill_top': 'Kill_top',f'{colName}_Kill_jgl': 'Kill_jgl',f'{colName}_Kill_mid': 'Kill_mid',f'{colName}_Kill_ad': 'Kill_ad', f'{colName}_Kill_sup': 'Kill_sup',
                                                  f'{colName}_Death_top': 'Death_top',f'{colName}_Death_jgl': 'Death_jgl',f'{colName}_Death_mid': 'Death_mid',f'{colName}_Death_ad': 'Death_ad',f'{colName}_Death_sup': 'Death_sup',
                                                  f'{colName}_Asisst_top': 'Assist_top',f'{colName}_Asisst_jgl': 'Assist_jgl',f'{colName}_Asisst_mid': 'Assist_mid',f'{colName}_Asisst_ad': 'Assist_ad',f'{colName}_Asisst_sup': 'Assist_sup',
                                                  f'{colName}_LV_top': 'LV_top',f'{colName}_LV_jgl': 'LV_jgl',f'{colName}_LV_mid': 'LV_mid',f'{colName}_LV_ad': 'LV_ad',f'{colName}_LV_sup': 'LV_sup',
                                                  f'{colName}_CS_top': 'CS_top',f'{colName}_CS_jgl': 'CS_jgl',f'{colName}_CS_mid': 'CS_mid',f'{colName}_CS_ad': 'CS_ad',f'{colName}_CS_sup': 'CS_sup',
                                                  f'{colName}_jglCS_top': 'jglCS_top',f'{colName}_jglCS_jgl': 'jglCS_jgl',f'{colName}_jglCS_mid': 'jglCS_mid',f'{colName}_jglCS_ad': 'jglCS_ad',f'{colName}_jglCS_sup': 'jglCS_sup',
                                                  f'{colName}_GOLD_top': 'GOLD_top',f'{colName}_GOLD_jgl': 'GOLD_jgl',f'{colName}_GOLD_mid': 'GOLD_mid',f'{colName}_GOLD_ad': 'GOLD_ad',f'{colName}_GOLD_sup': 'GOLD_sup',
                                                  f'{colName}_WARDkill': 'WARDkill',f'{colName}_Inhibitor': 'Inhibitor',f'{colName}_TOWERkill': 'TOWERkill',f'{colName}_WARDplaced': 'WARDplaced'})
win_Grandmaster['result'] = 1
lose_Grandmaster['result'] = -1
data = pd.concat([win_Grandmaster, lose_Grandmaster], axis=0, ignore_index=True)

##### 상관계수 높은 column 추출

In [51]:
# correlation_matrix = data.corr().round(2)
# Pass_Fail_corr = correlation_matrix['result']
# PFC=pd.DataFrame(Pass_Fail_corr)

# PFC_R = PFC.loc[abs(PFC.result) >= 0.1]
# corrData = data[PFC_R.index]
# data = corrData

##### 히트맵 그리기

In [52]:
# df = data
# columns = np.array(df.columns)
# df_small = df[columns]
# df_corr = df_small.corr()
# plt.figure(figsize=(50, 50))
# sns.heatmap(df_corr, annot=True, fmt=".2f", cmap="Blues")

In [17]:
rf = RandomForestClassifier(n_estimators=100, max_depth=50, random_state = 10)
lgbm = LGBMClassifier(n_estimators=100, verbosity=0)
cat = CatBoostClassifier(iterations=10, depth=2, learning_rate=1)
et = ExtraTreesClassifier(n_estimators=100, random_state = 10)


In [18]:
colCnt = data.shape[1]
print(colCnt)
X = data.iloc[:, :colCnt-1]
y = data.iloc[:, colCnt-1:]
# # features/target, train/test dataset 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할|
y_train = y_train.values.ravel()

48


In [55]:
rf.fit(X_train, y_train)
rf_pre = rf.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, rf_pre).ravel()
print("RandomForest :", round(accuracy_score(y_test, rf_pre)*100, 2), "%")
print("RandomForest f1 :", round(f1_score(y_test, rf_pre)*100, 2), "%")
print('tn:', tn, ' fp:', fp, ' fn:', fn, ' tp:', tp)
lgbm.fit(X_train, y_train)
lgbm_pre = lgbm.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, lgbm_pre).ravel()
print("lightGBM :", round(accuracy_score(y_test, lgbm_pre)*100, 2), "%")
print("lightGBM f1 :", round(f1_score(y_test, lgbm_pre)*100, 2), "%")
print('tn:', tn, ' fp:', fp, ' fn:', fn, ' tp:', tp)
cat.fit(X_train, y_train)
cat_pre = cat.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, cat_pre).ravel()
print("CatBoost :", round(accuracy_score(y_test, cat_pre)*100, 2), "%")
print("CatBoost f1 :", round(f1_score(y_test, cat_pre)*100, 2), "%")
print('tn:', tn, ' fp:', fp, ' fn:', fn, ' tp:', tp)
et = ExtraTreesClassifier(n_estimators=100, random_state = 10)
et.fit(X_train, y_train)
et_pre = et.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, et_pre).ravel()
print("ExtraTrees :", round(accuracy_score(y_test, et_pre)*100, 2), "%")
print("ExtraTrees f1 :", round(f1_score(y_test, et_pre)*100, 2), "%")
print('tn:', tn, ' fp:', fp, ' fn:', fn, ' tp:', tp)

RandomForest : 74.93 %
RandomForest f1 : 74.57 %
tn: 1007  fp: 318  fn: 343  tp: 969
lightGBM : 78.46 %
lightGBM f1 : 78.4 %
tn: 1038  fp: 287  fn: 281  tp: 1031
0:	learn: 0.6553607	total: 4.53ms	remaining: 40.7ms
1:	learn: 0.6229206	total: 12.9ms	remaining: 51.7ms
2:	learn: 0.6062569	total: 17.1ms	remaining: 39.9ms
3:	learn: 0.5967347	total: 21.8ms	remaining: 32.7ms
4:	learn: 0.5905871	total: 28.4ms	remaining: 28.4ms
5:	learn: 0.5830582	total: 34.6ms	remaining: 23.1ms
6:	learn: 0.5753948	total: 39.1ms	remaining: 16.7ms
7:	learn: 0.5717545	total: 44.6ms	remaining: 11.1ms
8:	learn: 0.5631357	total: 49.3ms	remaining: 5.47ms
9:	learn: 0.5583128	total: 53.3ms	remaining: 0us
CatBoost : 69.28 %
CatBoost f1 : 69.08 %
tn: 922  fp: 403  fn: 407  tp: 905
ExtraTrees : 77.66 %
ExtraTrees f1 : 77.54 %
tn: 1031  fp: 294  fn: 295  tp: 1017


In [25]:
column_names = ['Diff_FirstBLOOD', 'Diff_FirstDRAGON', 'Diff_FirstHERALD',
       'Diff_Firsttower', 'dragonType', 'invadeKill', 'invadeDeath',
       'controlWARDPlaced', 'Kill_top', 'Kill_jgl', 'Kill_mid', 'Kill_ad',
       'Kill_sup', 'Death_top', 'Death_jgl', 'Death_mid', 'Death_ad',
       'Death_sup', 'Assist_top', 'Assist_jgl', 'Assist_mid', 'Assist_ad',
       'Assist_sup', 'LV_top', 'LV_jgl', 'LV_mid', 'LV_ad', 'LV_sup', 'CS_top',
       'CS_jgl', 'CS_mid', 'CS_ad', 'CS_sup', 'jglCS_top', 'jglCS_jgl',
       'jglCS_mid', 'jglCS_ad', 'jglCS_sup', 'GOLD_top', 'GOLD_jgl',
       'GOLD_mid', 'GOLD_ad', 'GOLD_sup', 'WARDkill', 'Inhibitor', 'TOWERkill',
       'WARDplaced']
model = lgbm
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X):
    X_train, X_test = data.iloc[train_index][column_names], data.iloc[test_index][column_names]
    y_train, y_test = data.iloc[train_index]['result'], data.iloc[test_index]['result']
    model.fit(X_train, y_train)
    rf_pre = model.predict(X_test)

    # score = accuracy_score(y_test, rf_pre)
    print('accuracy_score : ', accuracy_score(y_test, rf_pre))
    print('f1_score : ', f1_score(y_test, rf_pre))
    tn, fp, fn, tp = confusion_matrix(y_test, rf_pre).ravel()
    print('tn:', tn, ' fp:', fp, ' fn:', fn, ' tp:', tp)
    print()
    

scores = cross_val_score(model, data[column_names], data['result'], cv=kf)
mean_score = np.mean(scores)
print(f'평균 평가 점수: {mean_score}')

accuracy_score :  0.7919666540356195
f1_score :  0.7890895121014215
tn: 1063  fp: 269  fn: 280  tp: 1027

accuracy_score :  0.7999242137173171
f1_score :  0.8026905829596412
tn: 1037  fp: 275  fn: 253  tp: 1074

accuracy_score :  0.8013646702047005
f1_score :  0.8038922155688623
tn: 1040  fp: 244  fn: 280  tp: 1074

accuracy_score :  0.8021228203184231
f1_score :  0.8053691275167785
tn: 1036  fp: 276  fn: 246  tp: 1080

accuracy_score :  0.7971948445792267
f1_score :  0.7977315689981095
tn: 1048  fp: 308  fn: 227  tp: 1055

평균 평가 점수: 0.7985146405710574


In [74]:
rf_pre

array([ 1,  1,  1, ..., -1, -1, -1], dtype=int64)

In [69]:
y_test

0        1
8        1
14       1
17       1
19       1
        ..
13143   -1
13153   -1
13166   -1
13180   -1
13181   -1
Name: result, Length: 2637, dtype: int64